---
### 🐣 초등학생을 위한 텍스트 전처리 친절 설명!

#### 📝 왜 텍스트를 전처리해요?

컴퓨터는 숫자만 이해해요! '안녕하세요'를 그대로 입력할 수 없어요.
텍스트를 숫자로 바꾸는 과정이 **전처리(Preprocessing)**예요!

#### 🔄 텍스트 → 숫자 변환 과정

```
원문: "나는 오늘 학교에 갔다"
    ↓ 토큰화(Tokenization)
["나는", "오늘", "학교에", "갔다"]
    ↓ 어휘사전 구축 (vocab)
{"나는":1, "오늘":2, "학교에":3, "갔다":4}
    ↓ 정수 인코딩
[1, 2, 3, 4]
    ↓ 패딩 (길이 맞추기)
[1, 2, 3, 4, 0, 0]  ← 길이 6으로 통일 (0=패딩)
```

#### 🌐 토큰화 방식 비교

| 방식 | 예시 | 특징 |
|------|------|------|
| **어절 단위** | "나는" "학교에" | 한국어에 자연스러움 |
| **자소 단위** | "ㄴ" "ㅏ" "ㄴ" "ㅡ" "ㄴ" | 매우 세밀함 |
| **서브워드** | "나" + "는" | 미등록 단어 처리 가능 |

> 💡 **패딩(Padding)이 왜 필요해요?**
> 문장마다 길이가 달라요. 배치로 처리하려면 모두 같은 길이여야 해요!
> 짧은 문장 뒤에 0(패딩 토큰)을 채워 길이를 맞춰요.


# Chapter 06-01: 텍스트 전처리

## 학습 목표
- 텍스트 데이터를 신경망에 입력하기 위한 전처리 과정을 이해한다.
- 토큰화(Tokenization)의 다양한 수준을 비교한다.
- TensorFlow의 `TextVectorization` 레이어를 활용한다.
- 패딩(Padding)과 마스킹(Masking)의 필요성을 이해한다.
- 한국어 텍스트의 특수성을 파악한다.

## 목차
1. [기본 임포트](#1.-기본-임포트)
2. [토큰화 수준 비교](#2.-토큰화-수준-비교)
3. [TextVectorization 레이어](#3.-TextVectorization-레이어)
4. [패딩과 마스킹](#4.-패딩과-마스킹)
5. [한국어 특수성](#5.-한국어-특수성)
6. [정리](#6.-정리)

In [ ]:
# 기본 라이브러리 임포트
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

# 한글 폰트 설정 (macOS)
matplotlib.rcParams['font.family'] = 'AppleGothic'
matplotlib.rcParams['axes.unicode_minus'] = False

print(f"TensorFlow 버전: {tf.__version__}")
print(f"NumPy 버전: {np.__version__}")

## 2. 토큰화 수준 비교

텍스트를 숫자로 변환하기 위해 먼저 **토큰(Token)**으로 분할해야 한다.  
토큰화 수준에 따라 아래와 같이 세 가지로 분류할 수 있다.

| 수준 | 예시 입력 | 토큰 결과 | 특징 |
|------|-----------|-----------|------|
| **단어 수준** | "I love NLP" | ["I", "love", "NLP"] | 직관적, 어휘 사전이 커짐, OOV 문제 |
| **서브워드 수준** | "I love NLP" | ["I", "love", "NL", "##P"] | OOV 완화, BERT 등에서 사용 (WordPiece, BPE) |
| **문자 수준** | "I love NLP" | ["I", " ", "l", "o", "v", "e", ...] | 어휘 사전 최소화, 시퀀스 매우 길어짐 |

### OOV(Out-Of-Vocabulary) 문제
학습 데이터에 없는 단어가 테스트 시 등장하는 문제이다.  
서브워드 방식은 단어를 더 작은 단위로 쪼개어 이 문제를 완화한다.  
예: "unhappiness" → ["un", "##happi", "##ness"]

## 3. TextVectorization 레이어

`tf.keras.layers.TextVectorization`은 원시 문자열 텍스트를  
정수 인덱스 시퀀스로 변환하는 전처리 레이어이다.

주요 단계:
1. **adapt()**: 훈련 데이터로부터 어휘 사전(vocabulary)을 구축한다.
2. **call()**: 텍스트를 정수 시퀀스로 변환한다.
3. **get_vocabulary()**: 구축된 어휘 사전을 반환한다.

특수 토큰:
- 인덱스 `0`: 패딩(padding) 토큰 `''`
- 인덱스 `1`: 미등록 단어(OOV) 토큰 `'[UNK]'`

In [ ]:
# TextVectorization 레이어 실습

# 샘플 텍스트 데이터
sample_texts = [
    "I love deep learning",
    "deep learning is amazing",
    "I love natural language processing",
    "natural language processing is fun",
]

# TextVectorization 레이어 생성
# max_tokens: 어휘 사전의 최대 크기 (None이면 무제한)
# output_sequence_length: 출력 시퀀스의 고정 길이 (None이면 가변)
vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=50,
    output_sequence_length=6,  # 모든 시퀀스를 길이 6으로 고정
    output_mode='int',         # 정수 인덱스 출력
)

# adapt(): 훈련 데이터로 어휘 사전 구축
vectorizer.adapt(sample_texts)

# 어휘 사전 확인
vocab = vectorizer.get_vocabulary()
print("어휘 사전 크기:", len(vocab))
print("어휘 사전 내용:", vocab)
print()

# 텍스트를 정수 시퀀스로 변환
test_input = ["I love learning", "unknown word test"]
encoded = vectorizer(test_input)
print("입력 텍스트:", test_input)
print("인코딩 결과:\n", encoded.numpy())
print()
print("인덱스 0: 패딩 토큰 ''") 
print("인덱스 1: OOV 토큰 '[UNK]' - 사전에 없는 단어")

## 4. 패딩(Padding)과 마스킹(Masking)

신경망은 고정 크기의 배치 입력을 요구한다.  
하지만 자연어 문장의 길이는 제각각이므로, **패딩**을 통해 길이를 맞춘다.

### 패딩 방식
- **post padding**: 시퀀스 뒤에 0을 채운다. `[1, 2, 3, 0, 0]` (기본값)
- **pre padding**: 시퀀스 앞에 0을 채운다. `[0, 0, 1, 2, 3]`

### 마스킹(Masking)
패딩된 0은 실제 의미 있는 데이터가 아니므로,  
모델이 이를 **무시**하도록 마스크를 사용한다.  
Embedding 레이어에서 `mask_zero=True`로 설정하면 자동으로 처리된다.

In [ ]:
# 패딩과 마스킹 실습

# 가변 길이 시퀀스 예시
sequences = [
    [1, 2, 3, 4, 5],      # 길이 5
    [1, 2, 3],             # 길이 3
    [1, 2, 3, 4, 5, 6, 7], # 길이 7
    [1],                   # 길이 1
]

# post 패딩 (기본값): 뒤에 0을 채움
padded_post = tf.keras.utils.pad_sequences(
    sequences,
    padding='post',   # 뒤에 0 추가
    truncating='post' # 최대 길이 초과 시 뒤를 자름
)
print("Post 패딩 결과:")
print(padded_post)
print()

# pre 패딩: 앞에 0을 채움
padded_pre = tf.keras.utils.pad_sequences(
    sequences,
    padding='pre',    # 앞에 0 추가
    truncating='pre'  # 최대 길이 초과 시 앞을 자름
)
print("Pre 패딩 결과:")
print(padded_pre)
print()

# mask_zero=True를 사용한 Embedding 레이어
# mask_zero=True: 패딩 토큰(0)을 무시하는 마스크를 자동 생성
embedding_with_mask = tf.keras.layers.Embedding(
    input_dim=10,
    output_dim=4,
    mask_zero=True  # 0 인덱스에 대한 마스크 생성
)

# 패딩된 시퀀스로 임베딩 수행
sample_input = tf.constant([[1, 2, 3, 0, 0]])  # 0은 패딩
output = embedding_with_mask(sample_input)

# 마스크 확인
mask = embedding_with_mask.compute_mask(sample_input)
print("입력 시퀀스:", sample_input.numpy())
print("마스크 (True=유효, False=패딩):", mask.numpy())

## 5. 한국어 특수성

### 교착어(Agglutinative Language)로서의 한국어

한국어는 **교착어**로, 어근에 다양한 접사가 붙어 의미와 문법 기능이 결정된다.  
따라서 단순히 공백으로 분리하면 같은 단어의 다양한 형태를 다른 단어로 인식한다.

예시: "먹다"의 다양한 형태
- 먹었습니다 = 먹(어간) + 었(과거 시제) + 습니다(공손 종결)
- 먹고 = 먹(어간) + 고(연결 어미)
- 먹어서 = 먹(어간) + 어서(이유 연결 어미)

공백 분리 시 "먹었습니다", "먹고", "먹어서"는 서로 다른 단어로 처리된다.  
형태소 분석을 사용하면 모두 어간 "먹"을 공유함을 파악할 수 있다.

### KoNLPy
한국어 형태소 분석을 위한 Python 라이브러리이다.  
Okt, Mecab, Komoran, Hannanum, Kkma 등 다양한 분석기를 지원한다.

In [ ]:
# 한국어 텍스트 처리 실습

# 교착어 예시: "먹다"의 다양한 활용형
korean_examples = [
    "나는 밥을 먹었습니다",
    "밥을 먹고 싶어요",
    "점심을 먹어서 배불러요",
    "저녁을 먹을 예정입니다",
]

print("=== 공백 기준 단순 분리 ===")
for text in korean_examples:
    tokens = text.split()
    # "먹"이 포함된 토큰 찾기
    eat_tokens = [t for t in tokens if '먹' in t]
    print(f"  입력: '{text}'")
    print(f"  토큰: {tokens}")
    print(f"  '먹' 포함 토큰: {eat_tokens}")
    print()

print("→ 공백 분리 시 '먹었습니다', '먹고', '먹어서', '먹을'이 모두 다른 토큰으로 처리됨")
print("→ 형태소 분석 시 모두 어간 '먹'을 공유함을 파악 가능")

# KoNLPy 설치 여부에 따른 처리
print("\n=== KoNLPy 형태소 분석 ===")
try:
    from konlpy.tag import Okt
    okt = Okt()
    
    for text in korean_examples:
        # morphs(): 형태소 단위로 분리
        morphs = okt.morphs(text)
        print(f"  입력: '{text}'")
        print(f"  형태소: {morphs}")
        print()
        
except ImportError:
    print("KoNLPy가 설치되지 않았습니다.")
    print("설치 방법: pip install konlpy")
    print()
    print("[시뮬레이션] KoNLPy Okt 형태소 분석 예상 결과:")
    
    # 형태소 분석 결과 시뮬레이션
    simulated_results = [
        ("나는 밥을 먹었습니다", ["나", "는", "밥", "을", "먹었", "습니다"]),
        ("밥을 먹고 싶어요",    ["밥", "을", "먹고", "싶어요"]),
    ]
    for text, morphs in simulated_results:
        print(f"  입력: '{text}'")
        print(f"  형태소: {morphs}")
        print()

## 6. 정리

### 핵심 개념 요약

| 개념 | 설명 | TensorFlow API |
|------|------|----------------|
| **토큰화** | 텍스트를 의미 단위로 분할 | `TextVectorization` |
| **어휘 사전** | 토큰과 인덱스의 매핑 | `adapt()`, `get_vocabulary()` |
| **OOV 처리** | 미등록 단어 → `[UNK]` (인덱스 1) | 자동 처리 |
| **패딩** | 시퀀스 길이를 맞추기 위해 0 추가 | `pad_sequences()` |
| **마스킹** | 패딩 토큰을 모델이 무시하도록 설정 | `mask_zero=True` |
| **형태소 분석** | 한국어 교착어 처리 | `KoNLPy` |

### 다음 챕터 예고
- **Chapter 06-02**: 단어 임베딩(Word Embeddings)  
  정수 인덱스를 의미 있는 연속 벡터로 변환하는 방법을 학습한다.